## Basic workflow for ARIMA model + update_predict()

In [35]:
import os
os.getcwd()

'/Users/eugenechua/code/chua0332/Hierarchical_Forecasting_Lewagon/notebooks'

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.series.date import DateTimeFeatures
from sktime.forecasting.base import ForecastingHorizon


In [39]:
data_reduced = pd.read_csv('/Users/eugenechua/code/chua0332/Hierarchical_Forecasting_Lewagon/data/train_reduced.csv', encoding='utf-8')

data_reduced.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
1,HOBBIES_1_058_CA_1_validation,HOBBIES_1_058,HOBBIES_1,HOBBIES,CA_1,CA,1,2,3,0,...,1,0,0,0,0,0,0,0,3,1
2,HOBBIES_1_062_CA_1_validation,HOBBIES_1_062,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
3,HOBBIES_1_082_CA_1_validation,HOBBIES_1_082,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,HOBBIES_1_104_CA_1_validation,HOBBIES_1_104,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,0,0,0,0,0,0,2


In [40]:
print(data_reduced.shape)

(1000, 1919)


Then we need to convert the data into the long format by using melt()

In [41]:
data_long = pd.melt(data_reduced,id_vars=data_reduced.columns[0:6],value_vars=data_reduced.columns[6:1920],var_name='days',value_name='value')

Then we can need to read in the calendar data and combine it with the original dataset.

In [42]:
calendar = pd.read_csv('/Users/eugenechua/code/chua0332/Hierarchical_Forecasting_Lewagon/data/calendar.csv')

calendar['date'] = pd.to_datetime(calendar['date'])

print(calendar.shape)
calendar.head()

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [43]:

data_long = data_long.merge(calendar, how='left', left_on='days', right_on='d')

In [44]:
data_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,days,value,date,wm_yr_wk,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_058_CA_1_validation,HOBBIES_1_058,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_062_CA_1_validation,HOBBIES_1_062,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_082_CA_1_validation,HOBBIES_1_082,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_104_CA_1_validation,HOBBIES_1_104,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0


In [45]:
data_long.dtypes

id                      object
item_id                 object
dept_id                 object
cat_id                  object
store_id                object
state_id                object
days                    object
value                    int64
date            datetime64[ns]
wm_yr_wk                 int64
weekday                 object
wday                     int64
month                    int64
year                     int64
d                       object
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                  int64
snap_TX                  int64
snap_WI                  int64
dtype: object

In [46]:
dat1 = data_long[['state_id','store_id','cat_id','dept_id','item_id','date','value']]
dat2 = dat1.sort_values(['state_id','store_id','cat_id','dept_id','item_id','date','value'])
dat3 = dat2.set_index(['state_id','store_id','cat_id','dept_id','item_id','date'])

dat3

value
state_id store_id cat_id    dept_id     item_id         date             
CA       CA_1     FOODS     FOODS_1     FOODS_1_018     2011-01-29     17
                                                        2011-01-30      3
                                                        2011-01-31      8
                                                        2011-02-01      2
                                                        2011-02-02      5
...                                                                   ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_515 2016-04-20      0
                                                        2016-04-21      1
                                                        2016-04-22      1
                                                        2016-04-23      1
                                                        2016-04-24      0

[1913000 rows x 1 columns]

In [47]:
pd.MultiIndex.from_frame(dat2)


MultiIndex([('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ...
            ('WI', 'WI_3', 'HOUSEHOLD', 'HOUSEHOLD_2', 'HOUSEHOLD_2_515', ...),
            ('WI', 'WI_3', 'HOUSEHOLD', 'HOUSEHOLD_2', 'HOUSEHOLD_2_515', ...),
            ('WI', 'WI_3

In [16]:
dat3

value
state_id store_id cat_id    dept_id     item_id         date             
CA       CA_1     FOODS     FOODS_1     FOODS_1_045     2011-01-29      0
                                                        2011-01-30      7
                                                        2011-01-31      0
                                                        2011-02-01      3
                                                        2011-02-02      0
...                                                                   ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_403 2016-04-20      0
                                                        2016-04-21      0
                                                        2016-04-22      0
                                                        2016-04-23      0
                                                        2016-04-24      0

[191300 rows x 1 columns]

In [48]:
from sktime.forecasting.arima import ARIMA

forecaster = ARIMA()

y_pred = forecaster.fit(dat3, fh=[1, 2]).predict()
y_pred

/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


value
state_id store_id cat_id    dept_id     item_id         date                 
CA       CA_1     FOODS     FOODS_1     FOODS_1_018     2016-04-25  11.250447
                                                        2016-04-26  10.431580
                                        FOODS_1_040     2016-04-25   1.724917
                                                        2016-04-26   2.018395
                                        FOODS_1_044     2016-04-25   2.007291
...                                                                       ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_425 2016-04-26   0.061578
                                        HOUSEHOLD_2_496 2016-04-25   0.112221
                                                        2016-04-26   0.120600
                                        HOUSEHOLD_2_515 2016-04-25   0.052769
                                                        2016-04-26   0.058395

[2000 rows x 1 columns]

This is the tricky bit ~ but lets just assume that y_test is the data that we are going to parse through to **update** the forecaster.

In [49]:
#Doing the train test split --> withholding 28 days for testing
y_train, y_test = temporal_train_test_split(dat3,test_size=28)

print(y_train.shape)
print(y_test.shape)

(1885000, 1)
(28000, 1)


In [50]:
y_test

value
state_id store_id cat_id    dept_id     item_id         date             
CA       CA_1     FOODS     FOODS_1     FOODS_1_018     2016-03-28      6
                                                        2016-03-29      4
                                                        2016-03-30      5
                                                        2016-03-31     15
                                                        2016-04-01     17
...                                                                   ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_515 2016-04-20      0
                                                        2016-04-21      1
                                                        2016-04-22      1
                                                        2016-04-23      1
                                                        2016-04-24      0

[28000 rows x 1 columns]

In [51]:
#checking/inspecting the index
y_test.index

MultiIndex([('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ('CA', 'CA_1',     'FOODS',     'FOODS_1',     'FOODS_1_018', ...),
            ...
            ('WI', 'WI_3', 'HOUSEHOLD', 'HOUSEHOLD_2', 'HOUSEHOLD_2_515', ...),
            ('WI', 'WI_3', 'HOUSEHOLD', 'HOUSEHOLD_2', 'HOUSEHOLD_2_515', ...),
            ('WI', 'WI_3

In [63]:
#In the hierarchical structure extract the dates in y_test for forecasting
dateindex = []
for i in list(y_test.index):
    dateindex.append(i[5])
    
dateindex    

[Timestamp('2016-03-28 00:00:00'),
 Timestamp('2016-03-29 00:00:00'),
 Timestamp('2016-03-30 00:00:00'),
 Timestamp('2016-03-31 00:00:00'),
 Timestamp('2016-04-01 00:00:00'),
 Timestamp('2016-04-02 00:00:00'),
 Timestamp('2016-04-03 00:00:00'),
 Timestamp('2016-04-04 00:00:00'),
 Timestamp('2016-04-05 00:00:00'),
 Timestamp('2016-04-06 00:00:00'),
 Timestamp('2016-04-07 00:00:00'),
 Timestamp('2016-04-08 00:00:00'),
 Timestamp('2016-04-09 00:00:00'),
 Timestamp('2016-04-10 00:00:00'),
 Timestamp('2016-04-11 00:00:00'),
 Timestamp('2016-04-12 00:00:00'),
 Timestamp('2016-04-13 00:00:00'),
 Timestamp('2016-04-14 00:00:00'),
 Timestamp('2016-04-15 00:00:00'),
 Timestamp('2016-04-16 00:00:00'),
 Timestamp('2016-04-17 00:00:00'),
 Timestamp('2016-04-18 00:00:00'),
 Timestamp('2016-04-19 00:00:00'),
 Timestamp('2016-04-20 00:00:00'),
 Timestamp('2016-04-21 00:00:00'),
 Timestamp('2016-04-22 00:00:00'),
 Timestamp('2016-04-23 00:00:00'),
 Timestamp('2016-04-24 00:00:00'),
 Timestamp('2016-03-

In [65]:
#Extracting the unique dates
dateindex = list(set(dateindex))
dateindex = pd.DatetimeIndex(dateindex)
dateindex = dateindex.sort_values()

In [68]:
#Creating the forecasting horizon!
fh = ForecastingHorizon(dateindex,is_relative=False)

Now we are predicting on the test data!

In [69]:
from sktime.forecasting.arima import ARIMA

forecaster = ARIMA()

forecaster.fit(y_train)
y_pred = forecaster.predict(fh)

y_pred

/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


value
state_id store_id cat_id    dept_id     item_id         date                 
CA       CA_1     FOODS     FOODS_1     FOODS_1_018     2016-03-28  12.660178
                                                        2016-03-29  11.100249
                                                        2016-03-30  10.371653
                                                        2016-03-31  10.031348
                                                        2016-04-01   9.872402
...                                                                       ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_515 2016-04-20   0.057292
                                                        2016-04-21   0.057292
                                                        2016-04-22   0.057292
                                                        2016-04-23   0.057292
                                                        2016-04-24   0.057292

[28000 rows x 1 columns]

### Ignore the reconciliaion method for now.

In [32]:
### Reconciliator Forecasting
from sktime.forecasting.reconcile import ReconcilerForecaster
from sktime.transformations.hierarchical.aggregate import Aggregator

agg = Aggregator()

#Fitting the aggregator to y_train
agg.fit_transform(y_train)
forecaster = ARIMA()
reconciler = ReconcilerForecaster(forecaster, method='bu')
reconciler.fit(y_train)

y_pred = reconciler.predict(fh)
y_pred

/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sktime/transformations/hierarchical/reconcile.py:295: PerformanceWarning: indexing past lexsort depth may impact performance.
  s_matrix.loc[tuple(agg_ind[::-1]), i] = 1.0


value
state_id store_id cat_id  dept_id item_id     date                 
CA       CA_1     FOODS   FOODS_1 FOODS_1_045 2016-03-28   1.709802
                                              2016-03-29   2.330937
                                              2016-03-30   2.556582
                                              2016-03-31   2.638554
                                              2016-04-01   2.668333
...                                                             ...
__total  __total  __total __total __total     2016-04-20  68.356509
                                              2016-04-21  68.356509
                                              2016-04-22  68.356509
                                              2016-04-23  68.356509
                                              2016-04-24  68.356509

[4312 rows x 1 columns]

### Now we will *mirror* the update_predict_single() workflow

In [70]:
#Fitting a base ARIMA model first
forecaster = ARIMA()

forecaster.fit(y_train)

/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ARIMA()

In [73]:
forecasting_horizon = pd.DatetimeIndex(pd.date_range(start='2016-04-25',end='2016-05-31',freq='D'))

In [74]:
#Assuming that y_test data is the new data coming in, we are then updating the forecaster to take it in and then project outwards
y_pred = forecaster.update_predict_single(y_test,fh=forecasting_horizon)

y_pred

/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/eugenechua/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
 

0
state_id store_id cat_id    dept_id     item_id         date                 
CA       CA_1     FOODS     FOODS_1     FOODS_1_018     2016-04-25  11.247481
                                                        2016-04-26  10.430020
                                                        2016-04-27  10.048716
                                                        2016-04-28   9.870858
                                                        2016-04-29   9.787896
...                                                                       ...
WI       WI_3     HOUSEHOLD HOUSEHOLD_2 HOUSEHOLD_2_515 2016-05-27   0.059597
                                                        2016-05-28   0.059597
                                                        2016-05-29   0.059597
                                                        2016-05-30   0.059597
                                                        2016-05-31   0.059597

[37000 rows x 1 columns]